## Building our own data Pipe lines 

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import numpy as np
from math import pi
import os
from math import pi
from urllib.request import urlopen
import matplotlib.patheffects as pe
from highlight_text import fig_text
from adjustText import adjust_text

import unicodedata
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [182]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [156]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    only_ascii = str(only_ascii)
    only_ascii = only_ascii[2:-1]
    only_ascii = only_ascii.replace('-', ' ')
    return only_ascii


In [151]:
url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data  = requests.get(url).text
soup = BeautifulSoup(data)
player_urls = []
links = BeautifulSoup(data).select('th a')
urls = [link['href'] for link in links]
urls = list(set(urls))
full_urls = []
for y in urls:
    full_url = "https://fbref.com"+y
    full_urls.append(full_url)
# full_urls

In [154]:
team_names = []
for team in urls: 
    team_name_slice = team[20:-6]
    team_names.append(team_name_slice)
list_of_tuples = list(zip(team_names, full_urls))
Team_url_database = pd.DataFrame(list_of_tuples, columns = ['team_names', 'urls'])


In [181]:
# for y in full_urls:
url = 'https://fbref.com/en/squads/cd051869/Brentford-Stats'

# print(url)
player_db = pd.DataFrame()
player_urls = []
data  = requests.get(url).text
links = BeautifulSoup(data).select('th a')
urls = [link['href'] for link in links]
player_urls.append(urls)
player_urls  = [item for sublist in player_urls  for item in sublist]
player_urls.sort()
player_urls = list(set(player_urls))
p_url = list(filter(lambda k: 'players' in k, player_urls))
url_final = []
for y in p_url:
    full_url = "https://fbref.com"+y
    url_final.append(full_url)
player_names = []
for player in p_url: 
    player_name_slice = player[21:]
    player_name_slice = player_name_slice.replace('-', ' ')
    player_names.append(player_name_slice)
# player_names
list_of_tuples = list(zip(player_names, url_final))
play_url_database = pd.DataFrame(list_of_tuples, columns = ['Player', 'urls'])
player_db = pd.concat([play_url_database])
# player_db 


        
# player_db

In [176]:
def general_url_database(full_urls):    
    appended_df = pd.DataFrame()
    for x in full_urls:
            url = x
            # print(url)
            player_db = pd.DataFrame()
            player_urls = []
            data  = requests.get(url).text
            links = BeautifulSoup(data).select('th a')
            urls = [link['href'] for link in links]
            player_urls.append(urls)
            player_urls  = [item for sublist in player_urls  for item in sublist]
            player_urls.sort()
            player_urls = list(set(player_urls))
            p_url = list(filter(lambda k: 'players' in k, player_urls))
            url_final = []
            for y in p_url:
                full_url = "https://fbref.com"+y
                url_final.append(full_url)
            player_names = []
            for player in p_url: 
                player_name_slice = player[21:]
                player_name_slice = player_name_slice.replace('-', ' ')
                player_names.append(player_name_slice)
            # player_names
            list_of_tuples = list(zip(player_names, url_final))
            play_url_database = pd.DataFrame(list_of_tuples, columns = ['Player', 'urls'])
            player_db = pd.concat([play_url_database])


            html = requests.get(x).text
            data = BeautifulSoup(html, 'html5')
            table = data.find('table')
            cols = []

            for header in table.find_all('th'):
                cols.append(header.string)

            columns = cols[8:37] #gets necessary column headers
            players = cols[37:-2]

            #display(columns)
            rows = [] #initliaze list to store all rows of data
            for rownum, row in enumerate(table.find_all('tr')): #find all rows in table
                if len(row.find_all('td')) > 0: 
                    rowdata = [] #initiliaze list of row data
                    for i in range(0,len(row.find_all('td'))): #get all column values for row
                        rowdata.append(row.find_all('td')[i].text)
                    rows.append(rowdata)
            df = pd.DataFrame(rows, columns=columns)

            df.drop(df.tail(2).index,inplace=True)
            df["Player"] = players
            df = df[["Player","Pos","Age", "Starts"]]

            df['Player'] = df.apply(lambda x: remove_accents(x['Player']), axis=1)
            test_merge = fuzzy_merge(df, player_db, 'Player', 'Player', threshold=97)
            test_merge = test_merge.rename(columns={'matches': 'Player', 'Player': 'matches'})
            test_merge = test_merge.drop(columns=['matches'])
            final_merge = test_merge.merge(player_db, on='Player', how='left')
    appended_df.append([final_merge])
    return appended_df 



In [177]:
general_url_database(full_urls)


""


In [180]:
appended_df = pd.DataFrame()
for x in full_urls:
        url = x
        # print(url)
        player_db = pd.DataFrame()
        player_urls = []
        data  = requests.get(url).text
        links = BeautifulSoup(data).select('th a')
        urls = [link['href'] for link in links]
        player_urls.append(urls)
        player_urls  = [item for sublist in player_urls  for item in sublist]
        player_urls.sort()
        player_urls = list(set(player_urls))
        p_url = list(filter(lambda k: 'players' in k, player_urls))
        url_final = []
        for y in p_url:
            full_url = "https://fbref.com"+y
            url_final.append(full_url)
        player_names = []
        for player in p_url: 
            player_name_slice = player[21:]
            player_name_slice = player_name_slice.replace('-', ' ')
            player_names.append(player_name_slice)
        # player_names
        list_of_tuples = list(zip(player_names, url_final))
        play_url_database = pd.DataFrame(list_of_tuples, columns = ['Player', 'urls'])
        player_db = pd.concat([play_url_database])


        html = requests.get(x).text
        data2 = BeautifulSoup(html, 'html5')
        table = data2.find('table')
        cols = []

        for header in table.find_all('th'):
            cols.append(header.string)

        columns = cols[8:37] #gets necessary column headers
        players = cols[37:-2]

        #display(columns)
        rows = [] #initliaze list to store all rows of data
        for rownum, row in enumerate(table.find_all('tr')): #find all rows in table
            if len(row.find_all('td')) > 0: 
                rowdata = [] #initiliaze list of row data
                for i in range(0,len(row.find_all('td'))): #get all column values for row
                    rowdata.append(row.find_all('td')[i].text)
                rows.append(rowdata)
        df = pd.DataFrame(rows, columns=columns)

        df.drop(df.tail(2).index,inplace=True)
        df["Player"] = players
        df = df[["Player","Pos","Age", "Starts"]]

        df['Player'] = df.apply(lambda x: remove_accents(x['Player']), axis=1)
        test_merge = fuzzy_merge(df, player_db, 'Player', 'Player', threshold=97)
        test_merge = test_merge.rename(columns={'matches': 'Player', 'Player': 'matches'})
        test_merge = test_merge.drop(columns=['matches'])
        final_merge = test_merge.merge(player_db, on='Player', how='left')
appended_df.append([final_merge])

,Pos,Age,Starts,Player,urls
0,GK,24-117,27,Robert Sanchez,https://fbref.com/en/players/6a713852/Robert-S...
1,DF,23-236,25,Marc Cucurella,https://fbref.com/en/players/1daec722/Marc-Cuc...
2,"FW,MF",27-101,24,Leandro Trossard,https://fbref.com/en/players/38ceb24a/Leandro-...
3,DF,30-059,23,Joel Veltman,https://fbref.com/en/players/dad4b285/Joel-Vel...
4,FW,25-213,22,Neal Maupay,https://fbref.com/en/players/4bcf39f6/Neal-Maupay
5,"MF,FW",22-342,19,Jakub Moder,https://fbref.com/en/players/f230bc30/Jakub-Moder
6,DF,30-114,19,Lewis Dunk,https://fbref.com/en/players/282f75f3/Lewis-Dunk
7,MF,25-197,18,Yves Bissouma,https://fbref.com/en/players/6c203af0/Yves-Bis...
8,MF,33-309,18,Adam Lallana,https://fbref.com/en/players/99813635/Adam-Lal...
9,"MF,DF",30-273,16,,NaN


In [ ]:
html = requests.get('https://fbref.com/en/squads/cd051869/Brentford-Stats').text
data = BeautifulSoup(html, 'html5')
table = data.find('table')
cols = []

for header in table.find_all('th'):
    cols.append(header.string)

columns = cols[8:37] #gets necessary column headers
players = cols[37:-2]

#display(columns)
rows = [] #initliaze list to store all rows of data
for rownum, row in enumerate(table.find_all('tr')): #find all rows in table
    if len(row.find_all('td')) > 0: 
        rowdata = [] #initiliaze list of row data
        for i in range(0,len(row.find_all('td'))): #get all column values for row
            rowdata.append(row.find_all('td')[i].text)
        rows.append(rowdata)
df = pd.DataFrame(rows, columns=columns)

df.drop(df.tail(2).index,inplace=True)
df["Player"] = players
df = df[["Player","Pos","Age", "Starts"]]